In [1]:
import pandas as pd
df_ana = pd.read_csv("../dados/df_condena_vazio.csv")

In [2]:
df_ana.columns

Index(['id', 'id_unique', 'numero_sei', 'ano_documento', 'assinaturas',
       'descricao_tipo_documento', 'descricao_tipo_processo',
       'numero_processo', 'id_unidade', 'sigla_unidade', 'descricao_unidade',
       'data_ordem', 'data_documento', 'setor_economico',
       'setor_economico_classe', 'setor_economico_divisao',
       'setor_economico_grupo', 'setor_economico_secao',
       'setor_economico_sub_classe', 'partes_processos_like',
       'partes_processos', 'decisao_sg', 'id_colecao', 'link_documento',
       'mercado_relevante', 'documentos_relacionados',
       'descricao_tipo_protocolo', 'descricao_especificacao', 'data_processo',
       '_version_', 'corpo_texto', 'decisao_tribunal', 'conteudo',
       'diferenca_dias'],
      dtype='object')

In [3]:
df_ana["decisao_tribunal"].value_counts()

decisao_tribunal
vazio    6756
Name: count, dtype: int64

In [4]:
df_ana20 = df_ana.head(20)

#### Regex

In [5]:
import re
def limpar_texto_cade(texto):
    
    # 1. Remove cabeçalhos e rodapés institucionais
    texto = re.sub(r'(?i)(timbre|ministério|conselho|endereço|telefone|cep|www\.).*?\n', '', texto)
    
    # 2. Remove URLs
    texto = re.sub(r'https?://\S+', '', texto)

    # 3. Remove expressões genéricas
    texto = re.sub(r'\[ACESSO RESTRITO\]', '', texto, flags=re.IGNORECASE)
    texto = re.sub(r'Brasília\s*,?\s*DF\s*-?\s*data da assinatura eletrônica.*?\n', '', texto, flags=re.IGNORECASE)
    texto = re.sub(r'\(sistema eletrônico\)', '', texto, flags=re.IGNORECASE)

    # 4. Remove notas de rodapé
    texto = re.sub(r'\[\d+\]', '', texto)

    # 5. Remove trechos longos de leis (mais agressivo)
    texto = re.sub(r'(Art\.?|artigo)\s+\d+.*?(Lei|Código|Decreto).*?(\n|$)', '(referência legal suprimida)', texto, flags=re.DOTALL)

    # 6. Remove números de página
    texto = re.sub(r'\n?\s*\d{1,3}\s*\n', '\n', texto)

    # 7. Remove datas formais
    texto = re.sub(r'\b\d{2}/\d{2}/\d{4}\b', '', texto)
    texto = re.sub(r'\b\d{4}-\d{2}-\d{2}\b', '', texto)

    # 8. Remove detalhes processuais irrelevantes
    texto = re.sub(r'SEI/CADE - \d+ - .*?\n', '', texto)
    texto = re.sub(r'Representante:.*?\n', '', texto)
    texto = re.sub(r'Representados?:.*?\n', '', texto)
    texto = re.sub(r'Relator\(a\)?:.*?\n', '', texto)

    # 9. Remove tabelas e quadros
    texto = re.sub(r'(\+\-+\+.*?\+\-+\+)', '', texto, flags=re.DOTALL)

    texto = re.sub(r'\|.*?\|', '', texto)
    
    # 10. Mantém apenas as seções críticas
    secoes_chave = re.findall(
        r'(?:CONCLUSÕES DA SUPERINTENDÊNCIA-GERAL|CONCLUSÕES DA PROCURADORIA|DO MÉRITO|DISPOSITIVO|voto pelo).*?(?:\n\n|$)',
        texto, 
        flags=re.DOTALL | re.IGNORECASE
    )
    
    texto = ' '.join(secoes_chave) if secoes_chave else texto

    # 11. Limpeza final
    texto = re.sub(r'\s{2,}', ' ', texto)
    texto = re.sub(r'\n{2,}', '\n', texto)
    texto = re.sub(r'(?i)(voto|decisão|condenação|arquivamento|multa)', '\n\\1', texto)  # Melhora estrutura
    
    return texto.strip()

In [6]:
import os, json
import pandas as pd
from dotenv import load_dotenv
from tqdm.auto import tqdm
import google.generativeai as genai

load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

# Configura o Gemini
genai.configure(api_key=API_KEY)
modelo = genai.GenerativeModel(
    "gemini-1.5-flash",
    generation_config={"temperature": 0.1}  
)

/home/gabriel/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def extrair_informacoes_juridicas(texto) :
    prompt = f"""
    Analise exclusivamente o DISPOSITIVO FINAL e CONCLUSÕES do documento jurídico do CADE abaixo.
    Ignore discussões processuais, evidências detalhadas ou fundamentação extensa.

    **Instruções específicas:**
    1. Para 'decisao_tribunal': considerar apenas os termos finais ("voto pelo arquivamento", "voto pela condenação")
    2. Para 'seguiu_nota_tecnica': verificar se o relator menciona explicitamente seguir ou divergir da nota técnica
    3. Para valores de multa: extrair apenas valores explícitos no dispositivo final
    4. Quando não encontrar informação clara, usar null

    Documento:
    \"\"\"{texto}\"\"\"

    Devolva APENAS JSON com os seguintes campos:
    {{
      "decisao_tribunal": "Condenação" | "Arquivamento" | null,
      "seguiu_nota_tecnica": true | false | null,
      "tipo_infracao_concorrencial": string | null,
      "multa": true | false,
      "tipo_de_multa": "valor_fixo" | "percentual" | "ambos" | null,
      "valor_multa_reais": float | null,
      "percentual_faturamento": float | null
    }}
    """.strip()

    resposta = modelo.generate_content(prompt)
    
    # Isola o JSON da resposta
    try:
        conteudo = resposta.text
        bloco_json = conteudo[conteudo.find('{'):conteudo.rfind('}')+1]
        return json.loads(bloco_json)
    except Exception as e:
        print(f"[ERRO ao parsear JSON]: {e}")
        print("Resposta bruta do Gemini:\n", resposta.text)
        return {
            "decisao_tribunal": None,
            "seguiu_nota_tecnica": None,
            "tipo_infracao_concorrencial": None,
            "multa": None,
            "tipo_de_multa": None,
            "valor_multa_reais": None,
            "percentual_faturamento": None
        }

In [8]:
def processar_linha(texto):
    texto_limpo = limpar_texto_cade(texto)
    return extrair_informacoes_juridicas(texto_limpo)

In [9]:
df_ana20['resultado'] = df_ana20['conteudo'].apply(processar_linha)

# Expande o dicionário da coluna 'resultado' em colunas separadas
df_expandido = pd.concat([df_ana20, df_ana20['resultado'].apply(pd.Series)], axis=1)

# Remove a coluna intermediária (opcional)
df_expandido.drop(columns=['resultado'], inplace=True)

# Mostra o DataFrame resultante
df_expandido

/tmp/ipykernel_6598/153724095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ana20['resultado'] = df_ana20['conteudo'].apply(processar_linha)


,id,id_unique,numero_sei,ano_documento,assinaturas,descricao_tipo_documento,descricao_tipo_processo,numero_processo,id_unidade,sigla_unidade,...,decisao_tribunal,conteudo,diferenca_dias,decisao_tribunal,seguiu_nota_tecnica,tipo_infracao_concorrencial,multa,tipo_de_multa,valor_multa_reais,percentual_faturamento
0,abbdc450-7ff1-4990-9bcc-8ae4386592f0,jurisprudencia__1390607,1390607,2024,Outras,Voto Processo Administrativo,Processo Administrativo,08700.004558/2019-05,110000967.0,GAB1,...,vazio,SEI/CADE - 1390607 - Voto Processo Administrat...,1714,Arquivamento,True,cartel,True,valor_fixo,324116.21,None
1,6b5bb7a0-9ad6-438f-96af-8eb3165fabaa,jurisprudencia__1389600,1389600,2024,Outras,Nota Técnica SG,Processo Administrativo,08700.004914/2021-05,110000983.0,CGAA7,...,vazio,SEI/CADE - 1389600 - Nota Técnica\n\n\nNota Té...,986,None,True,None,False,None,NaN,None
2,8a2483e2-0296-431e-85b2-11267caf35ec,jurisprudencia__1389754,1389754,2024,Outras,Parecer,Ato de Concentração Sumário,08700.002236/2024-81,110000980.0,CGAA5,...,vazio,SEI/CADE - 1389754 - Parecer\n\n\n\r\n\t\r\n\t...,43,Arquivamento,None,None,False,None,NaN,None
3,ba09f364-3ee5-4e28-bba3-c422d53b4ae2,jurisprudencia__1389765,1389765,2024,Outras,Parecer,Ato de Concentração Sumário,08700.002736/2024-12,110000980.0,CGAA5,...,vazio,SEI/CADE - 1389765 - Parecer\n\n\n\r\n\t\r\n\t...,27,Arquivamento,None,None,False,None,NaN,None
4,46dc8fec-b850-414f-babc-8cd0e1595ec5,jurisprudencia__1389787,1389787,2024,Outras,Parecer,Ato de Concentração Sumário,08700.002774/2024-75,110000980.0,CGAA5,...,vazio,SEI/CADE - 1389787 - Parecer\n\n\n\r\n\t\r\n\t...,25,None,None,None,False,None,NaN,None
5,e4ca50d2-f905-4a61-a2af-51eea9c56256,jurisprudencia__1389712,1389712,2024,Outras,Parecer,Ato de Concentração Sumário,08700.002810/2024-09,110000980.0,CGAA5,...,vazio,SEI/CADE - 1389712 - Parecer\n\n\n\r\n\t\r\n\t...,25,Arquivamento,None,None,False,None,NaN,None
6,64ffd058-0e27-4e39-a610-686f56eb48ed,jurisprudencia__1389690,1389690,2024,Outras,Parecer,Ato de Concentração Sumário,08700.002811/2024-45,110000980.0,CGAA5,...,vazio,SEI/CADE - 1389690 - Parecer\n\n\n\r\n\t\r\n\t...,25,Arquivamento,None,None,False,None,NaN,None
7,ea1d20dd-cd02-4d43-9045-5019ddc45ffb,jurisprudencia__1389716,1389716,2024,Outras,Parecer,Ato de Concentração Sumário,08700.002876/2024-91,110000980.0,CGAA5,...,vazio,SEI/CADE - 1389716 - Parecer\n\n\n\r\n\t\r\n\t...,22,Arquivamento,None,None,False,None,NaN,None
8,322ade44-7d28-4b76-8e09-4cabeb8151c3,jurisprudencia__1389735,1389735,2024,Outras,Parecer,Ato de Concentração Sumário,08700.002943/2024-77,110000980.0,CGAA5,...,vazio,SEI/CADE - 1389735 - Parecer\n\n\nPARECER Nº ...,21,Arquivamento,None,None,False,None,NaN,None
9,b8a0729b-b94a-4d4c-97ff-bbe4f0c44d88,jurisprudencia__1389730,1389730,2024,Outras,Parecer,Ato de Concentração Sumário,08700.002946/2024-19,110000980.0,CGAA5,...,vazio,SEI/CADE - 1389730 - Parecer\n\n\n\r\n\t\r\n\t...,21,Arquivamento,None,None,False,None,NaN,None
